In [1]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from konlpy.tag import Kkma
import re
from wordcloud import WordCloud
from wordcloud import STOPWORDS
import nltk

In [2]:
cpuData = pd.read_excel('./excel/cpuValidData.xlsx',  encoding = 'CP949')

In [6]:
for i in range(len(cpuData['대표청구항'])):
    cpuData['대표청구항'][i] = cpuData['대표청구항'][i][9:]
# cpuData['대표청구항']

In [9]:
for i in range(len(cpuData['발명의 명칭'])):
    cpuData['발명의 명칭'][i] = re.sub(r'\([^)]*\)', '', cpuData['발명의 명칭'][i])

In [11]:
cpuData_KR = cpuData[cpuData['국가코드'].isin(['KR']) | cpuData['국가코드'].isin(['JP'])]
cpuData_ENG = cpuData[cpuData['국가코드'].isin(['CN']) | cpuData['국가코드'].isin(['EP']) | cpuData['국가코드'].isin(['US'])]
print(len(cpuData_KR), len(cpuData_ENG))

1683 4314


In [12]:
cpuData_KR['kkk'] = cpuData_KR['발명의 명칭'] + ' ' + cpuData_KR['요약'] + cpuData_KR['대표청구항']
# cpuData_csv['꼬꼬마'] = cpuData_csv['발명의 명칭'][:6887] + ' ' + cpuData_csv['요약'][:6887] + cpuData_csv['대표청구항'][:6887]
# cpuData_csv['꼬꼬마'].drop(cpuData_csv['꼬꼬마'].index[6887:]) # 영어부분 삭제


# # cpuData_csv['nltk'] = cpuData_csv['발명의 명칭'][6887:] + ' ' + cpuData_csv['요약'][6887:] + cpuData_csv['대표청구항'][6887:]
# cpuData_csv['nltk'].drop(cpuData_csv['nltk'].index[:6887]) # 한글부분 삭제
cpuData_ENG['nltk'] = cpuData_ENG['발명의 명칭'] + ' ' + cpuData_ENG['요약'] + cpuData_ENG['대표청구항']

In [14]:
kkmaCPU = Kkma() # 꼬꼬마 객체
def morph(input_data):  # pos(모든 형태소 분석)
    preprcessed = kkmaCPU.pos(input_data)
    # print(preprcessed)
    return preprcessed

def get_wordData_KR(morpheme_pos):
    # words = [s for s, t in morpheme]
    # tags = [t for s, t in morpheme]
    wordData = []
    for words, tags in morpheme_pos:
        #if (tags == 'NNG') or (tags == 'NNP') or (tags == 'OL'):
        if (("NN") or ("OL")) in tags:
            wordData.append(words) # NNG, NNP, OL만 뽑음

    return wordData

def get_wordData_ENG(morpheme_pos):
    # words = [s for s, t in morpheme]
    # tags = [t for s, t in morpheme]
    wordData = []
    for words, tags in morpheme_pos:
        #if (tags == 'NNG') or (tags == 'NNP') or (tags == 'OL'):
        if (("NN")) in tags:
            wordData.append(words) # NNG, NNP, OL만 뽑음

    return wordData

In [15]:
%%time

# 영어 형태소 분석

nltk.download('punkt')
morph_ENG = [nltk.pos_tag(nltk.word_tokenize(x)) for x in cpuData_ENG['nltk'].values]
wordMaterialList_ENG = [get_wordData_ENG(ee) for ee in morph_ENG]

# nltk.download('punkt')
# morpheme_nltk_tokens = nltk.word_tokenize(str(cpuData_ENG['nltk'].values)[2:-3])
# morpheme_nltk = nltk.pos_tag(morpheme_nltk_tokens)

# # nltk.download('tagsets')
# wordMaterialList_ENG = get_wordData_ENG(morpheme_nltk)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Assistant_2\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Wall time: 1min 14s


In [17]:
%%time
# # 한글 형태소 분석
morph_KR = [morph(x) for x in cpuData_KR['kkk'].values]
wordMaterialList_KR = [get_wordData_KR(kk) for kk in morph_KR]

# morph(cpuData_KR['꼬꼬마'].values[0])
# t = map(lambda x: morph(x), list(cpuData_KR['꼬꼬마'].values))
# print(list(t))

# %%time


# morpheme_kkma = morph(str(cpuData_KR['꼬꼬마'].values)[2:-3])
# wordMaterialList_KR = get_wordData_KR(morpheme_kkma)

Wall time: 10min 28s


In [20]:
df_ENG = pd.DataFrame.from_records(wordMaterialList_ENG)
df_ENG.to_csv('./filtering_version/nounENG_csv.csv')
df_ENG.to_excel('./filtering_version/nounENG_excel.xlsx')

In [19]:
df_KR = pd.DataFrame.from_records(wordMaterialList_KR)
df_KR.to_csv('./filtering_version/nounKR_csv.csv')
df_KR.to_excel('./filtering_version/nounKR_excel.xlsx')